In [1]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import imageio
import numpy as np
import io
import helper 
import datetime
import os
from stable_baselines3 import PPO, DQN
import pandas as pd
import torch as th
from stable_baselines.bench import Monitor

In [2]:
###############################################################################
# Training params
###############################################################################
training_steps_exponent = 20 # Set to ~ 1mil
epoch_training_steps = 2**training_steps_exponent
epochs = 100
###############################################################################
# Evaluation params
###############################################################################
n_eval_episodes = 1000
###############################################################################
driving_agent = 'pelican'
config_fpath = '/data/configs/RBBRs_against_rulebased/monday_26x33_cl/monday_26x33_Panther_Agent_Move_North_pelican.json'

In [3]:
env = helper.get_env(driving_agent, config_fpath,
                    random_panther_start_position = True,
                    random_pelican_start_position = True,
                    max_illegal_moves_per_turn = 2, #self.max_illegal_moves_per_turn,
                    sparse = False,
                    curriculum_learning = True,
                    normalise = True)

env = Monitor(env, './logs')

policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[128, 64])

model = DQN('MlpPolicy',
            env, 
            policy_kwargs=policy_kwargs,                
            buffer_size=1000000,
            learning_starts=200,
            batch_size=128,
            gamma=0.95,
            #tau=0.0, # Soft target update uses: target_param.data.mul_(1 - tau) + ...
            target_update_interval=1000, # So every 10 - 20 Episodes
            exploration_fraction=.3, #exploration_fraction: fraction of entire training period over which the exploration rate is reduced
            exploration_initial_eps=1.0, # exploration_initial_eps: initial value of random action probability
            exploration_final_eps=0.1, # exploration_final_eps: final value of random action probability
            verbose=1)

model.learn(epoch_training_steps)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 1, win percentage 1.000000
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 2, win percentage 1.000000
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 3, win percentage 0.666667
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 4, win percentage 0.750000


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 29.5     |
|    ep_rew_mean      | 0.5      |
|    exploration rate | 1        |
| time/               |          |
|    episodes         | 4        |
|    fps              | 320      |
|    time_elapsed     | 0        |
|    total timesteps  | 118      |
----------------------------------


INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 5, win percentage 0.800000
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 6, win percentage 0.666667
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 7, win percentage 0.571429
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 8, win percentage 0.500000


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 36.8     |
|    ep_rew_mean      | 0        |
|    exploration rate | 0.999    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 94       |
|    time_elapsed     | 3        |
|    total timesteps  | 294      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0124   |
|    n_updates        | 23       |
----------------------------------


INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 9, win percentage 0.444444
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 10, win percentage 0.400000
INFO:gym_plark.envs.plark_env_cl:New Init Distance: 6, Wins: 0.400000 
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 1, win percentage 0.000000
INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 2, win percentage 0.000000


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 46       |
|    ep_rew_mean      | -0.333   |
|    exploration rate | 0.998    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 53       |
|    time_elapsed     | 10       |
|    total timesteps  | 552      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00424  |
|    n_updates        | 87       |
----------------------------------


INFO:gym_plark.envs.plark_env_cl:Len outcome tracker: 3, win percentage 0.333333


KeyboardInterrupt: 